In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import wikipediaapi as wiki
from datetime import datetime
import imdb
import random
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import wikipedia

In [ ]:
##########################
###     CONSTANTS      ###
##########################

In [33]:
# IMdb API key
api_key = "k_11m5rq35"

# headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive',
        "referer":"referer: https://www.google.com/",
        "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }

cookies = {"CONSENT":"YES+shp.gws-20210330-0-RC1.de+FX+412"}

# Parent and Relative strings to search for in wikipedia infobox
parent_pattern = re.compile('Parent')
relative_pattern = re.compile('Relative')
starring_pattern = re.compile('Starring')
voice_pattern = re.compile('Voices of')
wikipedia_pattern = re.compile("wiki")

not_nepo_relationships = 'son|daughter|nephew|niece|grandson|grand-daughter|grandnephew|grandniece|in-law|stepson|stepdaughter|brother|sister|cousin'

# url base
wiki_url_base = "https://en.wikipedia.org" # wikipedia
imdb_base_url = 'https://www.imdb.com/title/'

# wikipedia language setting
wiki_wiki = wiki.Wikipedia('en')

ia = imdb.Cinemagoer()

In [3]:
top_250_mov_url = "https://imdb-api.com/en/API/Top250Movies/"
top_250_tv_url = "https://imdb-api.com/en/API/Top250TVs/"
pop_mov_url = "https://imdb-api.com/en/API/MostPopularMovies/"
pop_tv_url = "https://imdb-api.com/en/API/MostPopularTVs/"
box_office_all_time_url = "https://imdb-api.com/en/API/BoxOfficeAllTime/"

In [ ]:
##########################
###     FUNCTIONS      ###
##########################

In [4]:
def get_imdb_lists(url, imdb_api_key):
    data = requests.get(url + imdb_api_key).json()
    list_of_titles = data['items']
    df = pd.DataFrame(list_of_titles)
    return df

In [11]:
df = get_imdb_lists(pop_mov_url, api_key)

In [75]:
def wiki_cast(title, year, tv_or_film): 
    print(title)
    title = title.replace(" ", "+").replace("&", "and")

    google_url = f"https://www.google.com/search?q={title}+{year}+{tv_or_film}+wikipedia"

    data_g = requests.get(google_url, headers=headers, cookies=cookies).text
    soup_g = BeautifulSoup(data_g,'html.parser') # full page

    links = []

    for link in soup_g.find_all('a'):
        l = link.get('href')
        links.append(l)
        
    # get only wikipedia links and get rid of image links
    # take first link (there are subsequent links for actors, other movies, etc)    
    if links is not None:
        links = [s for s in links if 'https://en.wikipedia.org/wiki/' in s and 'img' not in s][0]
    else :
        print("Links is none")
    
    # get the correct url
    wiki_url = re.search('url=(.*)&ved', links).group(1).replace('25', '')

    data = requests.get(wiki_url, headers=headers).text
    soup = BeautifulSoup(data,'html.parser') # full page

    # check if infobox exists
    infobox = soup.find("table",{"class":"infobox vevent"}) # infobox
    
    if infobox is None :
        return False # no infobox on wiki page -> not a nepo baby
        
    else :
        infobox_fields = infobox.find_all('th', {'class' : 'infobox-label'})

        does_starring_field_exist = bool(re.search("Starring", str(infobox_fields)))
        does_voices_field_exist = bool(re.search("Voices of", str(infobox_fields)))
        
        cast_list = []
        cast_links = []
        
        if does_voices_field_exist is False and does_starring_field_exist is False:
            cast_list = ["SKIP"]
            cast_links = ["SKIP"]
        else :
            if does_voices_field_exist is True:
                th_pattern = voice_pattern
            elif does_starring_field_exist is True:
                th_pattern = starring_pattern

            th = soup.find('th', string=th_pattern).parent
            li_list = th.find_all('li')

            # get cast names
            for li in li_list:
                a = li.find('a')
                if a == None :
                    cast_list.append(li.text)
                    cast_links.append("")
                else :
                    cast_list.append(a.text)
                    cast_links.append(a['href'])

            if len(cast_list) == 0 :
                cast_list = ["SKIP"]
                cast_links = ["SKIP"]
            else :
                cast_list
                cast_links

        return [cast_list, cast_links]

In [76]:
wiki_cast("The Proposal", "2009", "film")

The Proposal


TypeError: argument of type 'NoneType' is not iterable

In [6]:
def title_specs(title_id):
    print(title_id)
    page_data =  requests.get(imdb_base_url+title_id, headers=headers).text
    soup = BeautifulSoup(page_data,'html.parser')
    
    # CAST
    # grab top cast section of imdb page
    top_cast = soup.find_all("a", {"class": "sc-bfec09a1-1 gfeYgX"})
    
    cast_list = []
    cast_links = []
    
    # get cast names
    for a in top_cast:
        print(str(a.string))
        cast_list.append(str(a.string))
        cast_links.append("")
        
    return [cast_list, cast_links]

In [7]:
# given name of person, outputs parent links or false if none
def wiki_scrape(name, wiki_name_link=None):
    print(name)
    
    if name == 'SKIP' :
        return 'SKIP'
    
    if wiki_name_link != "" :
        url = wiki_url_base + wiki_name_link
        does_page_exist = True
    else :
        subject = name.replace(" ", "_")
        url = wiki_url_base + "/wiki/" + subject

        # check if wiki page exists
        wiki_page = wiki_wiki.page(subject)
        does_page_exist = wiki_page.exists()
        does_page_exist

    if does_page_exist is False :
        return False # no wiki page -> not famous enough, not a nepo baby
    
    elif does_page_exist is True : 
        data = requests.get(url, headers=headers).text
        soup = BeautifulSoup(data,'html.parser') # full page
        
        # check if infobox exists
        infobox = soup.find("table",{"class":"infobox biography vcard"}) # infobox
        
        if infobox is None :
            return False # no infobox on wiki page -> not a nepo baby
        
        else :
            infobox_fields = infobox.find_all('th', {'class' : 'infobox-label'})
            
            does_parent_or_rel_field_exist = bool(re.search("Parent|Relative", str(infobox_fields))) # see if Parent or Relative field is listed in infobox
        
            if does_parent_or_rel_field_exist is False :
                return False # parent field not listed in infobox -> not a nepo baby
            
            elif does_parent_or_rel_field_exist is True :
                
                try :
                    does_parent_field_exist = bool(re.search("Parent", str(infobox_fields)))
                    does_relative_field_exist = bool(re.search("Relative", str(infobox_fields)))
                    
                    if does_parent_field_exist is True :   
                        parent_field = soup.find('th', string=parent_pattern).parent
                        parent_a_tags = parent_field.find_all('a')
                        
                        if len(parent_a_tags) == 0 :
                            return False # parents listed in infobox but not linked -> not a nepo baby
                        
                        else : # nepo baby!
                            parent_wiki_list = []
                            
                            for link in parent_field.find_all('a'):
                                parent_wiki = link.get('href')
                                parent_wiki_link = wiki_url_base + parent_wiki
                                parent_wiki_list.append(parent_wiki_link)

                                parent_wiki_list[:] = [x for x in parent_wiki_list if "cite_note" not in x] # cited entries are in <a href> tags so remove those links here
                                
                        return parent_wiki_list

                    elif does_relative_field_exist is True : 
                        relative_field = soup.find('th', string=relative_pattern).parent
                        relative_td_tags = relative_field.find_all('td')

                        list_of_relatives = re.split('</li>|<br/>', str(relative_td_tags))

                        parent_wiki_list = []

                        if len(relative_field.find_all('a')) == 0 :
                            return False
                        else :
                            for i in range(len(list_of_relatives)):
                                if bool(re.search(not_nepo_relationships, list_of_relatives[i])) is True:
                                    pass

                                else :
                                    try :
                                        href_match = re.search(r'href=\"(.*)\" title=', list_of_relatives[i])
                                        parent_wiki = href_match.group(1)
                                        parent_wiki_link = wiki_url_base + parent_wiki
                                        parent_wiki_list.append(parent_wiki_link)

                                    except AttributeError :
                                        pass  
                            
                            if len(parent_wiki_list) == 0 :
                                return False
                            else :
                                return parent_wiki_list
                
                except AttributeError :
                    pass


In [8]:
def pct_nepo(df, group_col):
    df = df[[group_col, 'nepos']]
    df = df[df.nepos != False].groupby(group_col).count() / df.groupby(group_col).count()
    df = df.rename(columns={'nepos': 'pct_nepo'})
    return df

In [9]:
def imdb_whole_shebang(url, tv_or_film):
    
    # get list of titles from imdb
    df = get_imdb_lists(url, api_key) 
    # get cast list and cast wiki links from wikipedia
    df[['cast', 'cast_link']] = df.apply(lambda row : pd.Series(wiki_cast(row['title'], row['year'], tv_or_film)), axis=1)
    # remove rows where it has failed
    df = df[(df["cast"] != False)]
    # explode lists into rows
    df = df.explode(['cast', 'cast_link']).reset_index(drop=True) # expand list of cast into rows

    # find the nepos
    df.loc[:,"nepos"] = df.apply(lambda row : wiki_scrape(row['cast'], row['cast_link']), axis=1) # check if each cast member is a nepo
    
    # remove where the wiki_scrape hasn't worked
    noskip_df = df[df['cast'] != 'SKIP'] # no starring field
    nolist_df = noskip_df[~noskip_df['cast'].str.contains("list", case=False)] # cast list linked elsewhere - not in infobox
    correct_df = nolist_df[~nolist_df['cast'].str.contains("various", case=False)] # cast list linked elsewhere - not in infobox
    
    # find cast through imdb of the titles where wikipedia was insufficient
    fix_cast_imdb = df[(df['cast'] == 'SKIP') | df['cast'].str.contains("list", case=False) | df['cast'].str.contains("various", case=False)]
    fix_cast_imdb[['cast', 'cast_link']] = fix_cast_imdb.apply(lambda row : pd.Series(title_specs(row['id'])), axis=1) # get top cast from imdb
    fix_cast_imdb = fix_cast_imdb.explode(['cast', 'cast_link']).reset_index(drop=True) # expand list of cast into rows
    fix_cast_imdb.loc[:,"nepos"] = fix_cast_imdb.apply(lambda row : wiki_scrape(row['cast'], row['cast_link']), axis=1)
    
    # join the wiki cast and imdb cast df
    df = pd.concat([correct_df, fix_cast_imdb]).reset_index(drop=True)
    # drop duplicates incase they were not removed from correct_df. take last dupe as its the imdb list
    df.drop_duplicates(subset=['id','cast'], keep='last') 
    
    # calc percent nepo by title
    df_pct_nepo = pct_nepo(df, "id")
    df = df.merge(df_pct_nepo, on=['id'], how="left") # merge data frames
    
    # export
    csv_name_match = re.search('API/(.*)/', url)
    csv_name = csv_name_match.group(1)
    
    df.to_csv(csv_name + ".csv", index=False)
    
    return df


In [ ]:
df = imdb_whole_shebang(top_250_mov_url, "film")

The Shawshank Redemption
https://en.wikipedia.org/wiki/The_Shawshank_Redemption
The Godfather
https://en.wikipedia.org/wiki/The_Godfather
The Dark Knight
https://en.wikipedia.org/wiki/The_Dark_Knight
The Godfather Part II
https://en.wikipedia.org/wiki/The_Godfather_Part_II
12 Angry Men
https://en.wikipedia.org/wiki/12_Angry_Men_(1957_film)
Schindler's List
https://en.wikipedia.org/wiki/Schindler%27s_List
The Lord of the Rings: The Return of the King
https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Return_of_the_King
Pulp Fiction
https://en.wikipedia.org/wiki/Pulp_Fiction
The Lord of the Rings: The Fellowship of the Ring
https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Fellowship_of_the_Ring
The Good, the Bad and the Ugly
https://en.wikipedia.org/wiki/The_Good,_the_Bad_and_the_Ugly
Forrest Gump
https://en.wikipedia.org/wiki/Forrest_Gump
Fight Club
https://en.wikipedia.org/wiki/Fight_Club
The Lord of the Rings: The Two Towers
https://en.wikipedia.org/wiki/The_Lord_of_the_

Dangal
https://en.wikipedia.org/wiki/Dangal_(film)
For a Few Dollars More
https://en.wikipedia.org/wiki/For_a_Few_Dollars_More
Batman Begins
https://en.wikipedia.org/wiki/Batman_Begins
The Kid
https://en.wikipedia.org/wiki/The_Kid_(1921_film)
Some Like It Hot
https://en.wikipedia.org/wiki/Some_Like_It_Hot
The Father
https://en.wikipedia.org/wiki/The_Father_(2020_film)
All About Eve
https://en.wikipedia.org/wiki/All_About_Eve
The Wolf of Wall Street
https://en.wikipedia.org/wiki/The_Wolf_of_Wall_Street_(2013_film)
Green Book
https://en.wikipedia.org/wiki/Green_Book_(film)
Judgment at Nuremberg
https://en.wikipedia.org/wiki/Judgment_at_Nuremberg
Ran
https://en.wikipedia.org/wiki/Ran_(film)
Casino
https://en.wikipedia.org/wiki/Casino_(1995_film)
The Truman Show
https://en.wikipedia.org/wiki/The_Truman_Show
There Will Be Blood
https://en.wikipedia.org/wiki/There_Will_Be_Blood
Pan's Labyrinth
https://en.wikipedia.org/wiki/Pan%27s_Labyrinth
Unforgiven
https://en.wikipedia.org/wiki/Unforgiven

Dersu Uzala
https://en.wikipedia.org/wiki/Dersu_Uzala_(1975_film)
Dances with Wolves
https://en.wikipedia.org/wiki/Dances_with_Wolves
Tim Robbins
Morgan Freeman
Bob Gunton
William Sadler
Clancy Brown
Gil Bellows
James Whitmore
Marlon Brando
Al Pacino
James Caan
Richard Castellano
Robert Duvall
Sterling Hayden
John Marley
Richard Conte
Diane Keaton
Christian Bale
Michael Caine
Heath Ledger
Gary Oldman
Aaron Eckhart
Maggie Gyllenhaal
Morgan Freeman
Al Pacino
Robert Duvall
Diane Keaton
Robert De Niro
Talia Shire
Morgana King
John Cazale
Mariana Hill
Lee Strasberg
Henry Fonda
Lee J. Cobb
Ed Begley
E.G. Marshall
Jack Warden
Liam Neeson
Ben Kingsley
Ralph Fiennes
Caroline Goodall
Jonathan Sagall
Embeth Davidtz
Elijah Wood
Ian McKellen
Liv Tyler
Viggo Mortensen
Sean Astin
Cate Blanchett
John Rhys-Davies
Bernard Hill
Billy Boyd
Dominic Monaghan
Orlando Bloom
Hugo Weaving
Miranda Otto
David Wenham
Karl Urban
John Noble
Andy Serkis
Ian Holm
Sean Bean
John Travolta
Samuel L. Jackson
Uma Thurman
H

Elizabeth McGovern
Joe Pesci
Burt Young
Tuesday Weld
Treat Williams
Robin Williams
Matt Damon
Ben Affleck
Stellan Skarsgård
Minnie Driver
Ryunosuke Kamiki
Mone Kamishiraishi
Gene Kelly
Donald O'Connor
Debbie Reynolds
Jean Hagen
Millard Mitchell
Cyd Charisse
Aamir Khan
Kareena Kapoor
R. Madhavan
Sharman Joshi
Omi Vaidya
Boman Irani
Mona Singh
Javed Jaffrey
Ellen Burstyn
Jared Leto
Jennifer Connelly
Marlon Wayans
Christopher McDonald
Toshiro Mifune
Tatsuya Nakadai
Kyōko Kagawa
Tatsuya Mihashi
Yutaka Sada
Tom Hanks
Tim Allen
Joan Cusack
Don Rickles
Wallace Shawn
John Ratzenberger
Estelle Harris
Ned Beatty
Michael Keaton
Jodi Benson
John Morris
Zain Al Rafeea
Yordanos Shiferaw
Boluwatife Bankole
Kawthar Al Haddad
Fadi Kamel Youssef
Nour el Husseini
Alaa Chouchnieh
Cedra Izam
Nadine Labaki
Joseph Jimbazian
Farah Hasno
Boluwatife Treasure Bankole
Mark Hamill
Harrison Ford
Carrie Fisher
Billy Dee Williams
Anthony Daniels
David Prowse
Kenny Baker
Peter Mayhew
Frank Oz
Keir Dullea
Gary Lockwood

John Hurt
Jason Isaacs
Gary Oldman
Alan Rickman
Maggie Smith
David Thewlis
Julie Walters


In [ ]:
def how_many_nepos(title, year, tv_or_film):
    cast = wiki_cast(title, year, tv_or_film)
    df = pd.DataFrame(cast, columns=['cast'])
    df.loc[:,"nepos"] = df.apply(lambda row : wiki_scrape(row['cast']), axis=1)
    
    tot_cast = len(cast)
    num_nepos = len(df[(df['nepos']!=False)])
    
    nepos = df[(df['nepos']!=False)]['cast'].tolist()
    nepos_print = ', '.join(nepos)
    
    if num_nepos == 1 :
        verb = 'is a'
        plural = "y"
    else :
        verb = 'are'
        plural = "ies"    
    
    print(f'Out of the {tot_cast} top cast members, {num_nepos} {verb} nepo bab{plural}.')
    print(f"{nepos_print} {verb} nepo bab{plural}.")
                

In [ ]:
how_many_nepos('21 Jump Street', '2012', 'film')

In [ ]:
##########################
###   GET IMDB DATA    ###
##########################

In [ ]:
# top_250_mov = imdb_whole_shebang(top_250_mov_url, "film")
top_250_tv = imdb_whole_shebang(top_250_tv_url, "tv")
pop_mov = imdb_whole_shebang(pop_mov_url, "film")
pop_tv = imdb_whole_shebang(pop_tv_url, 'tv')
box_office = imdb_whole_shebang(box_office_all_time_url, "film")

In [ ]:
df = get_imdb_lists(pop_tv_url, api_key) # get list of titles from imdb

In [ ]:
df['cast'] = df.apply(lambda row : wiki_cast(row['title'], row['year'], "tv series"), axis=1)

In [ ]:
df = df[(df["cast"] != False)]

In [ ]:
df = df.explode('cast').reset_index(drop=True) # expand list of cast into rows

In [ ]:
df.loc[:,"nepos"] = df.apply(lambda row : wiki_scrape(row['cast']), axis=1)

In [ ]:
noskip_df = df[df['cast'] != 'SKIP']
nolist_df = noskip_df[~noskip_df['cast'].str.contains("list", case=False)]
correct_df = nolist_df[~nolist_df['cast'].str.contains("various", case=False)]

In [ ]:
fix_cast_imdb = df[(df['cast'] == 'SKIP') | df['cast'].str.contains("list", case=False) | df['cast'].str.contains("various", case=False)]
fix_cast_imdb.loc[:,"cast"] = fix_cast_imdb.apply(lambda row : title_specs(row['id']), axis=1) # get top cast from imdb
fix_cast_imdb = fix_cast_imdb.explode('cast').reset_index(drop=True) # expand list of cast into rows
fix_cast_imdb.loc[:,"nepos"] = fix_cast_imdb.apply(lambda row : wiki_scrape(row['cast']), axis=1)
fix_cast_imdb


In [ ]:
df = pd.concat([correct_df, fix_cast_imdb]).reset_index(drop=True)
df.drop_duplicates(subset=['id','cast'], keep='last')

In [ ]:
df_pct_nepo = pct_nepo(df, "id") # calculate pct nepo by title

In [ ]:
df = df.merge(df_pct_nepo, on=['id'], how="left") # merge data frames

In [ ]:
df.to_csv("poptv" + ".csv", index=False)

In [ ]:
##########################
###  DUPED NEPO ACTORS ###
##########################

In [ ]:
#cols_to_select = ['id', 'title', 'year', 'cast', 'nepos', 'pct_nepo']

# create one big fat df with all the diff imdb lists
# select desired cols
# top_250_mov_skinny = top_250_mov[cols_to_select]
# top_250_tv_skinny = top_250_tv[cols_to_select]
# box_office_skinny = box_office[cols_to_select]

# add column for type (tv or film)
top_250_mov['type'] = 'film'
top_250_tv['type'] = 'tv'
pop_mov['type'] = 'film'
pop_tv['type'] = 'tv'
#box_office_skinny['type'] = 'film'

all_df = [pop_mov, pop_tv]
df = pd.concat(all_df)

# drop actors+films groups that appear in >1 imdb list
df.drop_duplicates(subset=['id', 'cast'], keep=False)
df = df.drop(['crew', 'rank'], axis=1)
df['pct_nepo'] = df['pct_nepo'].replace(np.nan, 0) # no nepo babies in the top cast
df["imDbRatingCount"] = pd.to_numeric(df["imDbRatingCount"])

df.to_csv("all_imdb_data.csv", index=False)

### NEPOS ###
# create df with just nepo babies
nepos_df = df.loc[df['nepos'] != False]

# get count of how many titles a nepo baby appears in
nepos_num_titles = nepos_df.groupby(['cast']).size().reset_index(name='num_titles')
mean_nepos_repeats = nepos_num_titles['num_titles'].mean()

nepos_df.to_csv('nepos_only.csv', index=False)

### NOT NEPOS ###
non_nepo_df = df.loc[df['nepos'] == False]
non_nepo_num_titles = non_nepo_df.groupby(['cast']).size().reset_index(name='num_titles')
mean_non_nepo_repeats = non_nepo_num_titles['num_titles'].mean()

### OVERALL COUNTS ###
distinct_nepos = nepos_df.groupby(['cast'])
num_nepo_cast = distinct_nepos.ngroups

distinct_cast = df.groupby(['cast'])
num_cast = distinct_cast.ngroups

distinct_titles = df.groupby(['id'])
num_titles = distinct_titles.ngroups

stat_dict = {
    "total_cast": num_cast,
    "nepos_cast": num_nepo_cast,
    "num_titles": num_titles
    }

In [ ]:
##########################
###     USER INPUT     ###
##########################

In [ ]:
person_input = input("name: ")
person_cnxn = wiki_scrape(person_input)

if person_cnxn is False :
    print(f"{person_input} is not a nepo baby")
else :
    print(f'{person_input} is a nepo baby. Here are the connection links: {person_cnxn}')

In [ ]:
url_search_base = "https://imdb-api.com/en/API/SearchTitle/k_11m5rq35/"
movie_input = input("movie or tv show: ")

data = requests.get(url_search_base + movie_input).json()
data['results']


In [ ]:
movie_input = input("movie or tv show: ")
movie_choices = ia.search_movie(movie_input)

movie_choices_list = []

# only showing 5 possible options
for i in range(5) :
    
    index = [1, 2, 3, 4, 5]
    rank = index[i]
    
    movie_title = movie_choices[i]['title']
    movie_id = movie_choices[i].movieID
    
    movie = ia.get_movie(movie_id)
    
    short_cast_list = []
    try :
        cast = movie['cast']
        for i in range(3):
            short_cast_list.append(cast[i]['name'])
    
        formatted_cast_list = f"starring {short_cast_list[0]}, {short_cast_list[1]}, and {short_cast_list[2]}"
    except KeyError:
        formatted_cast_list = ""
    except IndexError:
        formatted_cast_list = ""
        
    try :
        year = movie['year']
        year = f" ({year})"
    except KeyError:
        year = ""
    print(f"{rank}: {movie_title}{year} {formatted_cast_list}")

movie_choice_input = input(f'people are not very creative with titles so there are a few titles - which number is the correct one?: ')

movie_id = movie_choices[int(movie_choice_input)-1].movieID
movie_url = imdb_base_url + "tt" + movie_id

movie_full_cast = title_specs("tt"+movie_id)

df = pd.DataFrame([movie_full_cast], columns=['title', 'cast', 'imdb_rating'])
df = df.explode('cast').reset_index(drop=True)
df.loc[:,"nepos"] = df.apply(lambda row : wiki_scrape(row['cast']), axis=1)
title_pct_nepo = pct_nepo(df, 'title')
pct_nepo_str = title_pct_nepo['pct_nepo'][0]

nepo_rows = df[df['nepos'] != False]

print(f'{movie_input} is {pct_nepo_str}% full of nepotism babies. The following people are the culprits: {nepo_rows}')
